---
# Initial Imports

In [ ]:
import os

print(os.getcwd())
os.chdir("../")
print(os.getcwd())

In [ ]:
import random
import time as t
from importlib import reload

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numba
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

from hypmm import build_model, centrality, centrality_utils, utils, weight_functions

In [ ]:
ptm_cmap = plt.cm.get_cmap("Reds")


def plot_PTM(ptm_arr, cmap, dis_cols, max_val, fontsize, annotate=True):

    # Plot heatmap of transition probabilities
    fig, ax = plt.subplots(1, 1, figsize=(12, 9))
    sns.heatmap(
        ptm_arr,
        cmap=cmap,
        ax=ax,
        linewidth=0.5,
        linecolor="k",
        vmax=max_val,
        cbar=True,
        annot=annotate,
        annot_kws={"fontsize": fontsize},
    )
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=fontsize)
    ax.set_yticklabels(dis_cols, fontsize=fontsize, rotation=0)
    ax.set_xticklabels(dis_cols, fontsize=fontsize, ha="center", rotation=90)
    ax.xaxis.tick_top()
    fig.tight_layout()

    return fig

In [ ]:
def plot_PR(
    pred_PR,
    succ_PR,
    dis_cols,
    vmax=1.0,
    offset=(-0.06, 0.01),
    bg_color=False,
    fontsize=20,
    add_text=False,
    colors=None,
):

    # Define colors
    N_diseases = len(dis_cols)
    nx, ny = offset
    if colors is None:
        colors = []
        for i in range(N_diseases):
            np.random.seed(i)
            colors.append(np.random.choice(range(156), size=3) / 255)

    # Define triangles
    L = vmax
    x = [L, L, 0, 0, 0.5 * L]
    y = [L, 0, 0, L, 0.5 * L]
    s = [0, -0.9 * L, 0, 0.9 * L, 0]
    triangles = [[2, 0, 1], [2, 0, 2], [2, 0, 3]]
    triang = tri.Triangulation(x, y, triangles)

    # Figure
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.axis([0, L, 0, L])
    ax.plot([0, L], [0, L], linewidth=3, color="k")
    if bg_color:
        ax.tripcolor(triang, s, alpha=0.3, cmap="bwr", shading="gouraud")
    ax.grid("on")

    # Annotate with disease titles
    dirhyp_pagerank = pred_PR.merge(succ_PR, on="Disease", how="inner")
    dirhyp_pagerank.columns = ["Disease", "Predecessor", "Successor"]
    dirhyp_pagerank_arr = dirhyp_pagerank[["Successor", "Predecessor"]].values
    for i, pts in enumerate(dirhyp_pagerank_arr):
        color = colors[i]
        ax.scatter(pts[0], pts[1], s=750, color=color)
        ax.text(pts[0] + nx, pts[1] + ny, s=dis_cols[i], color="k", fontsize=20)

    # Annotate figure labels, etc.
    if add_text:
        ax.text(
            0.3,
            0.35,
            "Transitive Conditions",
            fontsize=fontsize,
            rotation=45,
            transform=ax.transAxes,
        )
    ax.set_xlabel("Successor PageRank", fontsize=fontsize + 2)
    ax.set_ylabel("Predecessor PageRank", fontsize=fontsize + 2)
    ax.tick_params(axis="both", labelsize=fontsize)
    ax.set_aspect("equal", "box")
    fig.tight_layout()

    return fig

### Definitions from doc strings

`binmat (np.array, dtype=np.int8)`: 

Binary flag matrix whose test observations are represented by rows and diseases represented by columns.

`conds_worklist (np.array, dtype=np.int8)`: 

For each individual, the worklist stores the ordered set of conditions via their columns indexes. For compatability with Numba, once all conditions specified per individual, rest of row is a stream if -1's.

`idx_worklist (np.array, dtype=np.int8)`:

Worklist to specify any duplicates. For each individuals, if -2 then they only have 1 condition, if -1 then they have no duplicates and if they have positive integers indexing `conds_worklist` then it specifies which conditions
have a duplicate which needs taken care of.

### Example patients definition to input format

**Patients 1, 2 and 3:**
- Has diseases A, B, and C (i.e. `binmat: [1, 1, 1]`)
- Disease progression is A -> B -> C (i.e. `conds_worklist: [0, 1, 2]`)
- No duplicates are present (i.e. `idx_worklist: [-1, -1, -1]`)


**Patient 4:**
- Has diseases A, B, and C (i.e. `binmat: [1, 1, 1]`)
- Disease progression is C -> A -> B (i.e. `conds_worklist: [2, 0, 1]`)
- No duplicates are present (i.e. `idx_worklist: [-1, -1, -1]`)


**Patient 5:**
- Has diseases B and C (i.e. `binmat: [0, 1, 1]`)
- Disease progression is B -> C (i.e. `conds_worklist: [1, 2, -1]`)
- No duplicates are present (i.e. `idx_worklist: [-1, -1, -1]`)


**Patient 6:**
- Has disease B (i.e. `binmat: [0, 1, 0]`)
- No disease progression (just B) (i.e. `conds_worklist: [1, -1, -1]`)
- No duplicates are present, and only has one condition (i.e. `idx_worklist: [-2, -1, -1]`)


**Patient 7:**
- Has disease C (i.e. `binmat: [0, 0, 1]`)
- No disease progression (just C) (i.e. `conds_worklist: [2, -1, -1]`)
- No duplicates are present, and only has one condition (i.e. `idx_worklist: [-2, -1, -1]`)


**Patient 8:**
- Has diseases A, B and C (i.e. `binmat: [1, 1, 1]`)
- Disease progression is B -> A -> C (i.e. `conds_worklist: [1, 0, 2]`)
- No duplicates are present (i.e. `idx_worklist: [-1, -1, -1]`)


**Patient 9:**
- Has diseases A and B (i.e. `binmat: [1, 1, 0]`)
- Disease progression is B -> A (i.e. `conds_worklist: [1, 0, -1]`)
- No duplicates are present (i.e. `idx_worklist: [-1, -1, -1]`)


**Patient 10:**
- Has diseases A and C (i.e. `binmat: [1, 0, 1]`)
- Disease progression is A -> C (i.e. `conds_worklist: [0, 2, -1]`)
- No duplicates are present (i.e. `idx_worklist: [-1, -1, -1]`)

In [ ]:
# SYNTHETIC EXAMPLE FOR PAPER


binmat = np.array(
    [
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [0, 1, 1],
        [0, 1, 0],
        [0, 0, 1],
        [1, 1, 1],
        [1, 1, 0],
        [1, 0, 1],
    ],
    dtype=np.int8,
)

conds_worklist = np.array(
    [
        [0, 1, 2],
        [0, 1, 2],
        [0, 1, 2],
        [2, 0, 1],
        [1, 2, -1],
        [0, -1, -1],
        [2, -1, -1],
        [1, 0, 2],
        [0, 1, -1],
        [0, 2, -1],
    ],
    dtype=np.int8,
)

idx_worklist = np.array(
    [
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-2, -1, -1],
        [-2, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
    ],
    dtype=np.int8,
)

colarr = np.array(["A", "B", "C"], dtype="<U24")
n_diseases = colarr.shape[0]

print(binmat)
print(conds_worklist)

## Compute hyperedge and hyperarc weights

In [ ]:
inc_mat, weights, prev_arrs = build_model.compute_weights(
    binmat,
    conds_worklist,
    idx_worklist,
    colarr,
    "progression",
    weight_functions.modified_sorensen_dice_coefficient,
    utils.compute_progset,
    dice_type=1,
    plot=True,
    ret_inc_mat=None,
)

# Compute Successor and Predecessor Node PageRank Centrality

## Compute node and edge degree matrices

In [ ]:
inc_mat, weights, prev_arrs = build_model.compute_weights(
    binmat,
    conds_worklist,
    idx_worklist,
    colarr,
    "progression",
    weight_functions.modified_sorensen_dice_coefficient,
    utils.compute_progset,
    dice_type=1,
    plot=False,
    ret_inc_mat=None,
)

In [ ]:
hyperedge_weights = np.array(weights[0]["weight"])
hyperedge_titles = np.array(weights[0]["disease set"])

hyperarc_weights = np.array(weights[1]["weight"])
hyperarc_titles = np.array(weights[1]["progression"])

node_weights = np.array(weights[2]["weight"])
node_titles = np.array(weights[2]["node"])

In [ ]:
output = build_model.setup_vars(
    inc_mat, n_diseases, hyperarc_weights, hyperarc_titles, node_weights
)
inc_mat_data, hyperarc_data, node_weights, node_degs, edge_degs = output

inc_mat_tail, inc_mat_head = inc_mat_data
edge_weights, hyperarc_titles = hyperarc_data
node_degree_tail, node_degree_head = node_degs
edge_degree_tail, edge_degree_head = edge_degs

# Probability transition matrices

In [ ]:
P_node = centrality.comp_ptm_std(
    inc_mat_tail, inc_mat_head, edge_weights, node_degree_tail, None, "successor", eps=0
)

succ_ptm = plot_PTM(P_node, ptm_cmap, colarr, max_val=0.95, fontsize=36)

In [ ]:
P_node = centrality.comp_ptm_std(
    inc_mat_tail,
    inc_mat_head,
    edge_weights,
    node_degree_head,
    edge_degree_tail,
    "predecessor",
    eps=0,
)

pred_ptm = plot_PTM(P_node, ptm_cmap, colarr, max_val=0.95, fontsize=36)

# PageRank

In [ ]:
inpt = (
    (inc_mat_tail, inc_mat_head),
    (edge_weights, node_weights),
    node_degs,
    edge_degree_tail,
)
node_pagerank = centrality.pagerank_centrality(
    inpt,
    rep="standard",
    typ="successor",
    tolerance=1e-6,
    max_iterations=1000,
    is_irreducible=False,
    weight_resultant=False,
    random_seed=None,
    eps=0.0001,
)

all_node_sc_evc = pd.DataFrame(
    {
        "Disease": colarr,
        "PageRank Centrality": node_pagerank,
    }
)
print(f"PageRank sum: {node_pagerank.sum()}")
succ_order = (
    all_node_sc_evc.sort_values(by="PageRank Centrality", ascending=False)
    .reset_index(drop=True)
    .Disease
)
succ_node_sc_evc = (
    all_node_sc_evc.sort_values(by="PageRank Centrality", ascending=False)
    .reset_index(drop=True)
    .round({"PageRank Centrality": 3})
)

succ_node_sc_evc

In [ ]:
inpt = (
    (inc_mat_tail, inc_mat_head),
    (edge_weights, node_weights),
    node_degs,
    edge_degree_tail,
)
node_pagerank = centrality.pagerank_centrality(
    inpt,
    rep="standard",
    typ="predecessor",
    tolerance=1e-6,
    max_iterations=100,
    is_irreducible=False,
    weight_resultant=False,
    random_seed=None,
    eps=0.0001,
)

all_node_pr_evc = pd.DataFrame(
    {
        "Disease": colarr,
        "PageRank Centrality": node_pagerank,
    }
)
print(f"PageRank sum: {node_pagerank.sum()}")
pred_order = (
    all_node_pr_evc.sort_values(by="PageRank Centrality", ascending=False)
    .reset_index(drop=True)
    .Disease
)
pred_node_sc_evc = (
    all_node_pr_evc.sort_values(by="PageRank Centrality", ascending=False)
    .reset_index(drop=True)
    .round({"PageRank Centrality": 3})
)

pred_node_sc_evc

## Successor-Predecessor PageRank Visualisation

In [ ]:
colors = ["r", "g", "b"]
PR_fig = plot_PR(
    pred_node_sc_evc,
    succ_node_sc_evc,
    colarr,
    colors=colors,
    add_text=True,
    bg_color=True,
    vmax=1.0,
)

---
# Test to compute equally likely disease transitions among conditions

In [ ]:
# PREPARED EXAMPLE TO GENERATE EQUALLY LIKELY PAGERANK

binmat = np.array(
    [
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
    ],
    dtype=np.int8,
)

conds_worklist = np.array(
    [
        [0, 1, 2],
        [0, 2, 1],
        [1, 2, 0],
        [1, 0, 2],
        [2, 0, 1],
        [2, 1, 0],
        [0, 1, 2],
        [0, 2, 1],
        [1, 2, 0],
        [1, 0, 2],
        [2, 0, 1],
        [2, 1, 0],
    ],
    dtype=np.int8,
)

idx_worklist = np.array(
    [
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
    ],
    dtype=np.int8,
)

colarr = np.array(["A", "B", "C"], dtype="<U24")
n_diseases = colarr.shape[0]

print(binmat)
print(conds_worklist)

In [ ]:
inc_mat, weights, prev_arrs = build_model.compute_weights(
    binmat,
    conds_worklist,
    idx_worklist,
    colarr,
    "progression",
    weight_functions.modified_sorensen_dice_coefficient,
    utils.compute_progset,
    dice_type=1,
    plot=True,
    ret_inc_mat=None,
)
weights

In [ ]:
hyperedge_weights = np.array(weights[0]["weight"])
hyperedge_titles = np.array(weights[0]["disease set"])

hyperarc_weights = np.array(weights[1]["weight"])
hyperarc_titles = np.array(weights[1]["progression"])

node_weights = np.array(weights[2]["weight"])
node_titles = np.array(weights[2]["node"])

In [ ]:
output = build_model.setup_vars(
    inc_mat, n_diseases, hyperarc_weights, hyperarc_titles, node_weights
)
inc_mat_data, hyperarc_data, node_weights, node_degs, edge_degs = output

inc_mat_tail, inc_mat_head = inc_mat_data
edge_weights, hyperarc_titles = hyperarc_data
node_degree_tail, node_degree_head = node_degs
edge_degree_tail, edge_degree_head = edge_degs

# Probability transition matrices

In [ ]:
P_node = centrality.comp_ptm_std(
    inc_mat_tail, inc_mat_head, edge_weights, node_degree_tail, None, "successor", eps=0
)

succ_ptm = plot_PTM(P_node, ptm_cmap, colarr, max_val=0.95, fontsize=36)

In [ ]:
P_node = centrality.comp_ptm_std(
    inc_mat_tail,
    inc_mat_head,
    edge_weights,
    node_degree_head,
    edge_degree_tail,
    "predecessor",
    eps=0,
)

pred_ptm = plot_PTM(P_node, ptm_cmap, colarr, max_val=0.95, fontsize=36)

In [ ]:
inpt = (
    (inc_mat_tail, inc_mat_head),
    (edge_weights, node_weights),
    node_degs,
    edge_degree_tail,
)
node_pagerank = centrality.pagerank_centrality(
    inpt,
    rep="standard",
    typ="successor",
    tolerance=1e-6,
    max_iterations=1000,
    is_irreducible=False,
    weight_resultant=False,
    random_seed=None,
    eps=0.0001,
)

all_node_sc_evc = pd.DataFrame(
    {
        "Disease": colarr,
        "PageRank Centrality": node_pagerank,
    }
)
print(f"PageRank sum: {node_pagerank.sum()}")
succ_order = (
    all_node_sc_evc.sort_values(by="PageRank Centrality", ascending=False)
    .reset_index(drop=True)
    .Disease
)
succ_node_sc_evc = (
    all_node_sc_evc.sort_values(by="PageRank Centrality", ascending=False)
    .reset_index(drop=True)
    .round({"PageRank Centrality": 3})
)

succ_node_sc_evc

In [ ]:
inpt = (
    (inc_mat_tail, inc_mat_head),
    (edge_weights, node_weights),
    node_degs,
    edge_degree_tail,
)
node_pagerank = centrality.pagerank_centrality(
    inpt,
    rep="standard",
    typ="predecessor",
    tolerance=1e-6,
    max_iterations=100,
    is_irreducible=False,
    weight_resultant=False,
    random_seed=None,
    eps=0.0001,
)

all_node_pr_evc = pd.DataFrame(
    {
        "Disease": colarr,
        "PageRank Centrality": node_pagerank,
    }
)
print(f"PageRank sum: {node_pagerank.sum()}")
pred_order = (
    all_node_pr_evc.sort_values(by="PageRank Centrality", ascending=False)
    .reset_index(drop=True)
    .Disease
)
pred_node_sc_evc = (
    all_node_pr_evc.sort_values(by="PageRank Centrality", ascending=False)
    .reset_index(drop=True)
    .round({"PageRank Centrality": 3})
)

pred_node_sc_evc

In [ ]:
colors = ["r", "g", "b"]
PR_fig = plot_PR(
    pred_node_sc_evc,
    succ_node_sc_evc,
    colarr,
    colors=colors,
    add_text=True,
    bg_color=True,
    vmax=1.0,
)